## 李子桐 2019111002

a. Because sometimes it cannot be said that A and B are related, and it cannot be said that the occurrence of A has an impact on B. For example, when PB is originally high, the probability of B occurrence when A occurs is inherently high, and it cannot be explained that they are related. . LIFT and CONVICTION can explain the relevance of A and B. If lift=1, A and B are independent, lift<1, then the probability of A occurring when B occurs, and lift>1, then the probability of A occurring when B occurs Will improve. Conv=1, when A and B are independent, when conv>1, when B occurs, the probability of A will decrease; when conv<1, when B occurs, the probability of A will increase.

b.Confidence is asymmetric
Con(A-B)=support(AB)/support(A)
Con(B-A)=support(AB)/support(B)
If P(A)=/P(B) then asymmetric

Lift is symmetric
Lift(A-B)=support(AB)/support(A)*support(B)
Lift(B-A)=support(AB), support(B)*support(A)
Commutative law of multiplication

Conviction is asymmetric
Conv(A-B)=Nsupport(A)-support(A)*support(B)/support(A)-support(AB)
Conv(B-A)=Nsupport(B)-support(B)*support(A)/support(B)-support(BA)
Support(AB)!=support(A)*support(B)
So asymmetric

c.Conviction has this property
When conf=1, A must happen when B occurs, at this time lift=1/support(B),conviction=1-support(B)/0. When support tends to 0, the denominator of lift gradually tends to 0, and the denominator of conviction is 0, so conviction is larger at this time. When support tends to 1, lift gradually tends to 1, and in this process conviction always tends to positive infinity. In summary, conviction is the largest, so conviction has this property.

The codes for d and e are below

In [1]:
from pyspark import SparkContext

DEBUG = 1


def Dprint(info):
    if DEBUG:
        print(info)


def generate_next_c(f_k, k):
    next_c = [var1 | var2 for index, var1 in enumerate(f_k) for var2 in f_k[index + 1:] if
              list(var1)[:k - 2] == list(var2)[:k - 2]]
    return next_c


def generate_f_k(sc, c_k, shared_itemset, sup):
    def get_sup(x):
        x_sup = len([1 for t in shared_itemset.value if x.issubset(t)])
        if x_sup >= sup:
            return x, x_sup
        else:
            return ()

    f_k = sc.parallelize(c_k).map(get_sup).filter(lambda x: x).collect()
    return f_k


def apriori(sc, f_input, min_sup, max_k):
    # read the raw data
    data = sc.textFile(f_input)
    # count the total number of samples
    n_samples = data.count()
    # min_sup to frequency
    sup = min_sup
    # split sort
    itemset = data.map(lambda line: sorted(list(set([item for item in line.strip("\n").strip().split(' ')]))))
    # share the whole itemset with all workers
    shared_itemset = sc.broadcast(itemset.collect())
    # store for all freq_k
    frequent_itemset = []
    # prepare candidate_1
    k = 1
    c_k = itemset.flatMap(lambda x: x).distinct().collect()
    c_k = [{x} for x in c_k]

    # when candidate_k is not empty
    while len(c_k) > 0 and k <= max_k:
        # generate freq_k
        # Dprint("C{}: {}".format(k, c_k))
        f_k = generate_f_k(sc, c_k, shared_itemset, sup)
        # Dprint("F{}: {}".format(k, f_k))
        print(k, " : ", len(f_k))
        frequent_itemset.append(f_k)
        # generate candidate_k+1
        k += 1
        c_k = generate_next_c([set(item) for item in map(lambda x: x[0], f_k)], k)
    sc.stop()
    return frequent_itemset


def get_three_item_conf(frequent_itemset):

    dic_lst = []

    for item_lst in frequent_itemset:
        dic = {}
        for item, val in item_lst:
            dic[",".join(sorted(list(item)))] = val
        dic_lst.append(dic)

    frequent2 = dic_lst[1]
    three_item_conf_dic = {}
    for item, cnt in frequent_itemset[2]:
        first_item, second_item, third_item = item
        first_second_cnt = frequent2[",".join(sorted([first_item, second_item]))]
        first_third_cnt = frequent2[",".join(sorted([first_item, third_item]))]
        second_third_cnt = frequent2[",".join(sorted([second_item, third_item]))]

        three_item_conf_dic["%s->%s"%(str({first_item, second_item}), str({third_item}))] = cnt/first_second_cnt
        three_item_conf_dic["%s->%s"%(str({first_item, third_item}), str({second_item}))] = cnt/first_third_cnt
        three_item_conf_dic["%s->%s"%(str({second_item, third_item}), str({first_item}))] = cnt/second_third_cnt
    return three_item_conf_dic


def get_two_item_conf(frequent_itemset):

    dic_lst = []

    for item_lst in frequent_itemset:
        dic = {}
        for item,val in item_lst:
            dic[str(item)] = val
        dic_lst.append(dic)

    frequent1 = dic_lst[0]
    two_item_conf_dic = {}
    for item, cnt in frequent_itemset[1]:
        first_item, second_item = item
        first_cnt = frequent1[str({first_item})]
        second_cnt = frequent1[str({second_item})]
        two_item_conf_dic["%s->%s"%({first_item}, {second_item})] = cnt/first_cnt
        two_item_conf_dic["%s->%s"%({second_item}, {first_item})] = cnt/second_cnt
    return two_item_conf_dic


if __name__ == "__main__":
    input = "C:/Users/Administrator/Desktop/DS2/browsing_new.txt"
    frequent_itemset = apriori(SparkContext(appName="Spark Apriori"), input, 100, 3)
    two_item_conf_dic = get_two_item_conf(frequent_itemset)
    three_item_conf_dic =get_three_item_conf(frequent_itemset)
    two_item_conf_lst = sorted(two_item_conf_dic.items(), key=lambda kv:(kv[1], kv[0]), reverse=True)
    three_item_conf_lst = sorted(three_item_conf_dic.items(), key=lambda kv:(kv[1], kv[0]), reverse=True)
    print(two_item_conf_lst[0:5])
    print(three_item_conf_lst[0:5])

1  :  579
2  :  1104
3  :  216
[("{'DAI93865'}->{'FRO40251'}", 1.0), ("{'GRO85051'}->{'FRO40251'}", 0.9991554054054054), ("{'ELE12951'}->{'FRO40251'}", 0.9901960784313726), ("{'FRO92469'}->{'FRO40251'}", 0.9886075949367089), ("{'DAI88079'}->{'FRO40251'}", 0.9875930521091811)]
[("{'SNA80324', 'GRO85051'}->{'FRO40251'}", 1.0), ("{'SNA45677', 'GRO85051'}->{'FRO40251'}", 1.0), ("{'GRO73461', 'GRO85051'}->{'FRO40251'}", 1.0), ("{'GRO38814', 'GRO85051'}->{'FRO40251'}", 1.0), ("{'GRO21487', 'GRO85051'}->{'FRO40251'}", 1.0)]
